In [5]:
#!pip install pandas-datareader

#import sys
#!{sys.executable} -m pip install pmdarima

#!conda config --add channels conda-forge
#!conda config --set channel_priority strict
#!conda install pmdarima

  Using cached pmdarima-2.0.2-cp38-cp38-win_amd64.whl (571 kB)
  Using cached statsmodels-0.13.5-cp38-cp38-win_amd64.whl (9.2 MB)
  Using cached packaging-23.0-py3-none-any.whl (42 kB)
  Using cached patsy-0.5.3-py2.py3-none-any.whl (233 kB)
  Attempting uninstall: patsy
    Found existing installation: patsy 0.5.1
    Uninstalling patsy-0.5.1:
      Successfully uninstalled patsy-0.5.1
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\LUIS\\anaconda3\\Lib\\site-packages\\~tatsmodels\\nonparametric\\linbin.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



^C


In [1]:
# Librerias #

# Loading Libraries
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from scipy import stats
from pandas_datareader import data

import statsmodels.api as sm
#import pmdarima as pmd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.linear_model import Ridge, Lasso, ElasticNet, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)

# MODULO 1: INTRODUCCION A PYTHON y MODELOS UNIVARIADOS

### I.2. Base de datos con mas de una variable 

En esta seccion mostraremos una manera de obtener mas de una variable a la vez.
Aqui se requiere que los datos esten en la misma frecuencia (mes, trimestre, etc.)

In [2]:
# Ejemplo 1: Obtener una variable a la vez
url_base  = 'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/'
cod_var   = ['PD04692MD','PD31893DD','PD04637PD','PD04638PD'] 
form_out  = '/json'
period    = '/2003-01-02/2019-12-31'

df1 = pd.DataFrame()

for j in cod_var:
    url_aux   = url_base + j + form_out + period
    resp      = requests.get(url_aux)
    resp_json = resp.json()
    periods   = resp_json['periods']
    
    value = []
    dates = []
    
    for i in periods:
        aux_dat = i['name']
        aux_val = i['values']
        dates.append(aux_dat)
        value.append(float(aux_val[0]))
    
    dict_aux = {'Fecha' : dates, 
                 resp_json['config']['series'][0]['name'] : value}
    df_aux = pd.DataFrame(dict_aux)
    
    df_aux.set_index(df_aux['Fecha'], inplace=True)
    df_aux = df_aux.drop(columns=['Fecha'])
    df1    = pd.concat([df1, df_aux], axis=1)

display(df1)
df1.to_csv('FrecuenciaDiariaAntigua.csv')



,"Tasas de interés - Interbancaria, S/",Tasas de interés - Rendimiento del Bono del gobierno peruano a 10 años (en S/),Tipo de cambio - TC Interbancario (S/ por US$) - Compra,Tipo de cambio - TC Interbancario (S/ por US$) - Venta
Fecha,,,,
02.Ene.03,3.50,NaN,3.5052,3.5071
03.Ene.03,3.55,NaN,3.4985,3.5002
06.Ene.03,3.55,NaN,3.4997,3.5012
07.Ene.03,3.60,NaN,3.4953,3.4966
08.Ene.03,3.86,NaN,3.4910,3.4923
...,...,...,...,...
30.Ago.16,NaN,5.670,NaN,NaN
01.Nov.16,NaN,5.949,NaN,NaN
17.Nov.16,NaN,6.614,NaN,NaN
